In [1]:
# Install libraries
# !pip install psutil
# !pip install tqdm
# !pip install statsmodels
# !pip install scipy

In [5]:
import os
import glob
import pandas as pd
import psutil
from tqdm import tqdm


- This project will focus solely on datasets named in the format **dot1_MMYY**, **dot2_MMYY**, and **dot3_MMYY**, where the suffix represents the month and year.

- Since the data is organized into five main folders (**2020**, **2021**, **2022**, **2023**, and **2024**), each containing subfolders for the months with their respective datasets, a **for loop** will be used to iterate through them for processing.
- #### Estimating Optimal Chunk Size

To efficiently process large CSV files, a small sample of 1,000 rows is read to estimate the average memory usage per row. The script then calculates 10% of the system’s available RAM and divides it by the average row size to determine the optimal `chunksize` — the number of rows that can be safely processed at a time without overloading memory.

```python
chunksize = int(memory_budget / row_size)


In [12]:
import os
import glob
import pandas as pd
import psutil
from tqdm import tqdm

# 1. Calculate Optimal Chunk Size Based on Available Memory

sample_file = r"C:\Users\kpbon\Desktop\Azubi Africa\Project 1\data\2020\April2020TransBorderRawData\Apr 2020\dot1_0420.csv"
sample_df = pd.read_csv(sample_file, nrows=1000)  # Load a sample to estimate row size
row_size = sample_df.memory_usage(deep=True).sum() / 1000  # Average size per row (bytes)
memory_budget = psutil.virtual_memory().available * 0.1  # Use 10% of available RAM
chunksize = int(memory_budget / row_size)  # Rows per chunk

print('------------------------------------------------------')
print(f"Using chunk size: {chunksize}")
print('------------------------------------------------------')

# 2. Setup Main Folder and Dataset Storage

data_folder = r'C:\Users\kpbon\Desktop\Azubi Africa\Project 1\data'
data_dict = {'dot1': [], 'dot2': [], 'dot3': []}  # Store combined chunks by type

# 3. Month Name to Month Number Mapping

month_mapping = {
    "Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04", "May": "05", "Jun": "06",
    "Jul": "07", "Aug": "08", "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"
}

# 4. Loop Through All Year Folders (2020–2024)

for year in tqdm(['2020', '2021', '2022', '2023', '2024'], desc="Processing Years"):
    year_path = os.path.join(data_folder, year)

    # Loop through wrapper folders like "April2020TransBorderRawData"
    for wrapper_folder in tqdm(os.listdir(year_path), desc=f"Processing folders in {year}"):
        wrapper_path = os.path.join(year_path, wrapper_folder)

        # Skip if it's not a directory
        if not os.path.isdir(wrapper_path):
            continue

        # Loop through actual month folders like "Apr 2020"
        for month_folder in os.listdir(wrapper_path):
            month_path = os.path.join(wrapper_path, month_folder)

            # Extract month and year
            if " " not in month_folder:
                continue  # Skip folders not in expected format

            month_name, folder_year = month_folder.split(" ")
            month_number = month_mapping.get(month_name[:3])
            file_suffix = f"_{month_number}{year[-2:]}"  # E.g., _0420

            # 5. Identify Files with Expected Names and Extensions

            pattern = os.path.join(month_path, f"dot*{file_suffix}.*")
            all_files = glob.glob(pattern)
            target_files = [
                f for f in all_files
                if os.path.basename(f) in [
                    f"dot1{file_suffix}.csv", f"dot2{file_suffix}.csv", f"dot3{file_suffix}.csv",
                    f"dot1{file_suffix}.xlsx", f"dot2{file_suffix}.xlsx", f"dot3{file_suffix}.xlsx",
                    f"dot1{file_suffix}.xls",  f"dot2{file_suffix}.xls",  f"dot3{file_suffix}.xls"
                ]
            ]

            # 6. Read Each File by Chunk and Store in Corresponding Type

            for file in tqdm(target_files, desc=f"Reading files in {month_folder}"):
                dataset_type = os.path.basename(file).split('_')[0]

                try:
                    if file.endswith('.csv'):
                        for chunk in pd.read_csv(file, chunksize=chunksize):
                            data_dict[dataset_type].append(chunk)
                    elif file.endswith(('.xlsx', '.xls')):
                        df = pd.read_excel(file)
                        data_dict[dataset_type].append(df)
                except Exception as e:
                    print(f"Error reading file {file}: {e}")

# 7. Concatenate All Chunks into Final DataFrames

all_dot1 = pd.concat(data_dict['dot1'], ignore_index=True) if data_dict['dot1'] else pd.DataFrame()
all_dot2 = pd.concat(data_dict['dot2'], ignore_index=True) if data_dict['dot2'] else pd.DataFrame()
all_dot3 = pd.concat(data_dict['dot3'], ignore_index=True) if data_dict['dot3'] else pd.DataFrame()

# 8. Print Final Dataset Shapes

print('------------------------------------------------------')
print('All dot1 shape:', all_dot1.shape)
print('All dot2 shape:', all_dot2.shape)
print('All dot3 shape:', all_dot3.shape)
print('------------------------------------------------------')


------------------------------------------------------
Using chunk size: 533942
------------------------------------------------------


Processing folders in 2020:   0%|                                                                | 0/9 [00:00<?, ?it/s]

Reading files in Apr 2020:   0%|                                                                 | 0/3 [00:00<?, ?it/s]

Reading files in Apr 2020: 100%|█████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 10.76it/s]

Processing folders in 2020:  11%|██████▏                                                 | 1/9 [00:00<00:02,  3.39it/s]

Reading files in Feb 2020:   0%|                                                                 | 0/3 [00:00<?, ?it/s]C:\Users\kpbon\AppData\Local\Temp\ipykernel_356\500587952.py:76: DtypeWarning: Columns (4,11) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file, chunksize=chunksize):


Reading files in Feb 2020: 100%|█████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  9.62it/s]

Processing folders in 2020:  33%|██████████████████

------------------------------------------------------
All dot1 shape: (214171, 14)
All dot2 shape: (585849, 14)
All dot3 shape: (135189, 12)
------------------------------------------------------


### DOT1: Dataset Summary
This section provides a concise overview of the DOT1 dataset, including:
- The number of rows and columns
- Column data types
- Memory usage
- Non-null counts per column


In [19]:
print("\n========== DOT1 Dataset Summary ==========")
print(all_dot1.info())



========== DOT1 Dataset Summary ==========
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214171 entries, 0 to 214170
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   TRDTYPE          214171 non-null  int64  
 1   USASTATE         214171 non-null  object 
 2   DEPE             214171 non-null  object 
 3   DISAGMOT         214171 non-null  int64  
 4   MEXSTATE         61209 non-null   object 
 5   CANPROV          136751 non-null  object 
 6   COUNTRY          214171 non-null  int64  
 7   VALUE            214171 non-null  int64  
 8   SHIPWT           214171 non-null  int64  
 9   FREIGHT_CHARGES  214171 non-null  int64  
 10  DF               133167 non-null  float64
 11  CONTCODE         214171 non-null  object 
 12  MONTH            214171 non-null  int64  
 13  YEAR             214171 non-null  int64  
dtypes: float64(1), int64(8), object(5)
memory usage: 22.9+ MB
None


In [31]:
print("\n========== DOT2 Dataset Summary ==========")
print(all_dot2.info())



========== DOT2 Dataset Summary ==========
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 585849 entries, 0 to 585848
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   TRDTYPE          585849 non-null  int64  
 1   USASTATE         585849 non-null  object 
 2   COMMODITY2       585849 non-null  int64  
 3   DISAGMOT         585849 non-null  int64  
 4   MEXSTATE         196421 non-null  object 
 5   CANPROV          353611 non-null  object 
 6   COUNTRY          585849 non-null  int64  
 7   VALUE            585849 non-null  int64  
 8   SHIPWT           585849 non-null  int64  
 9   FREIGHT_CHARGES  585849 non-null  int64  
 10  DF               406105 non-null  float64
 11  CONTCODE         585849 non-null  object 
 12  MONTH            585849 non-null  int64  
 13  YEAR             585849 non-null  int64  
dtypes: float64(1), int64(9), object(4)
memory usage: 62.6+ MB
None


In [33]:
print("\n========== DOT3 Dataset Summary ==========")
print(all_dot3.info())



========== DOT3 Dataset Summary ==========
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135189 entries, 0 to 135188
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   TRDTYPE          135189 non-null  int64  
 1   DEPE             135189 non-null  object 
 2   COMMODITY2       135189 non-null  int64  
 3   DISAGMOT         135189 non-null  int64  
 4   COUNTRY          135189 non-null  int64  
 5   VALUE            135189 non-null  int64  
 6   SHIPWT           135189 non-null  int64  
 7   FREIGHT_CHARGES  135189 non-null  int64  
 8   DF               80752 non-null   float64
 9   CONTCODE         135189 non-null  object 
 10  MONTH            135189 non-null  int64  
 11  YEAR             135189 non-null  int64  
dtypes: float64(1), int64(9), object(2)
memory usage: 12.4+ MB
None


### UNDERSTANDING THE DATASET

DOT 1, DOT 2, and DOT 3 have different sets of columns. However, they all share the following 10 columns:  
**TRDTYPE**, **DISAGMOT**, **COUNTRY**, **VALUE**, **SHIPWT**, **FREIGHT_CHARGES**, **DF**, **CONTCODE**, **MONTH**, and **YEAR**.

In addition:
- **DOT 1** and **DOT 2** have **three additional columns** in common: **USASTATE**, **MEXSTATE**, and **CANPROV**.
- **DOT 1** and **DOT 3** share **one additional column**: **DEPE**.
- **DOT 2** and **DOT 3** share **one additional column**: **COMMODITY2**.

- **Note**: This will result in null values for the columns that are not in all of them when the data is merged


### Missing Values Check
This step helps identify columns with missing data and how many null values each contains.


In [25]:
print("\nMissing values in dot1:")
print(all_dot1.isnull().sum())



Missing values in dot1:
TRDTYPE                 0
USASTATE                0
DEPE                    0
DISAGMOT                0
MEXSTATE           152962
CANPROV             77420
COUNTRY                 0
VALUE                   0
SHIPWT                  0
FREIGHT_CHARGES         0
DF                  81004
CONTCODE                0
MONTH                   0
YEAR                    0
dtype: int64


In [27]:
print("\nMissing values in dot2:")
print(all_dot2.isnull().sum())



Missing values in dot2:
TRDTYPE                 0
USASTATE                0
COMMODITY2              0
DISAGMOT                0
MEXSTATE           389428
CANPROV            232238
COUNTRY                 0
VALUE                   0
SHIPWT                  0
FREIGHT_CHARGES         0
DF                 179744
CONTCODE                0
MONTH                   0
YEAR                    0
dtype: int64


In [29]:
print("\nMissing values in dot3:")
print(all_dot3.isnull().sum())


Missing values in dot3:
TRDTYPE                0
DEPE                   0
COMMODITY2             0
DISAGMOT               0
COUNTRY                0
VALUE                  0
SHIPWT                 0
FREIGHT_CHARGES        0
DF                 54437
CONTCODE               0
MONTH                  0
YEAR                   0
dtype: int64


In [50]:
merged_data = pd.concat([all_dot1, all_dot2, all_dot3], ignore_index = True)

In [51]:
merged_data.shape

(935209, 15)

In [54]:
merged_data.head()

,TRDTYPE,USASTATE,DEPE,DISAGMOT,MEXSTATE,CANPROV,COUNTRY,VALUE,SHIPWT,FREIGHT_CHARGES,DF,CONTCODE,MONTH,YEAR,COMMODITY2
0,1,AK,0115,5,NaN,XB,1220,4660,0,67,2.0,X,4,2020,NaN
1,1,AK,0901,5,NaN,XO,1220,14360,0,282,1.0,X,4,2020,NaN
2,1,AK,20XX,1,XX,NaN,2010,4293733,24971000,0,1.0,0,4,2020,NaN
3,1,AK,20XX,3,NaN,XA,1220,28283,443,563,1.0,X,4,2020,NaN
4,1,AK,20XX,3,NaN,XA,1220,29848,69,538,2.0,X,4,2020,NaN


In [57]:
merged_data['CONTCODE'].unique()

array(['X', '0', '1', 0, 1], dtype=object)

### Interpretation of `CONTCODE` (Container Code)

- `'X'` - **Containerized**  
- `'0'` - **Non-Containerized**  
- `'1'` - **Unknown** (no official match found, assumed based on absence of definition)

To make the data more meaningful during analysis, I assigned:
- `'X'` as **"Containerized"**
- `'0'` as **"Non-Containerized"**
- `'1'` as **"Unknown"**, since it had no documented explanation


In [59]:
merged_data['CONTCODE'] = merged_data['CONTCODE'].replace({'X': 'Containerized', '0': 'Non-Containerized',
                                                         0: 'Non-Containerized', '1': 'Unknown', 1: 'Unknown'})

In [61]:
merged_data['CONTCODE'].unique()

array(['Containerized', 'Non-Containerized', 'Unknown'], dtype=object)

In [73]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 935209 entries, 0 to 935208
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   TRDTYPE          935209 non-null  int64  
 1   USASTATE         800020 non-null  object 
 2   DEPE             349360 non-null  object 
 3   DISAGMOT         935209 non-null  int64  
 4   MEXSTATE         257630 non-null  object 
 5   CANPROV          490362 non-null  object 
 6   COUNTRY          935209 non-null  int64  
 7   VALUE            935209 non-null  int64  
 8   SHIPWT           935209 non-null  int64  
 9   FREIGHT_CHARGES  935209 non-null  int64  
 10  DF               620024 non-null  float64
 11  CONTCODE         935209 non-null  object 
 12  MONTH            935209 non-null  int64  
 13  YEAR             935209 non-null  int64  
 14  COMMODITY2       721038 non-null  float64
dtypes: float64(2), int64(8), object(5)
memory usage: 107.0+ MB


In [71]:
merged_data.nunique()

TRDTYPE                 2
USASTATE               52
DEPE                  241
DISAGMOT                8
MEXSTATE               33
CANPROV                14
COUNTRY                 2
VALUE              430405
SHIPWT             153131
FREIGHT_CHARGES     86711
DF                      2
CONTCODE                3
MONTH                   7
YEAR                    2
COMMODITY2             98
dtype: int64

In [75]:
merged_data.isnull().sum()

TRDTYPE                 0
USASTATE           135189
DEPE               585849
DISAGMOT                0
MEXSTATE           677579
CANPROV            444847
COUNTRY                 0
VALUE                   0
SHIPWT                  0
FREIGHT_CHARGES         0
DF                 315185
CONTCODE                0
MONTH                   0
YEAR                    0
COMMODITY2         214171
dtype: int64

### MISSING VALUES EXPLAINED

As predicted earlier, missing values have been introduced due to the merging of datasets with different column structures. Columns that are not present in all datasets appear as null in the combined dataset. This is expected and reflects the original layout of the individual files.

In addition, some columns—though present across all datasets—may still contain missing values within them. These gaps could result from incomplete data entries in the source files and are not caused by the merging process.

Overall, the presence of missing values is both structural and data-driven, and it will be addressed appropriately during the data cleaning phase.


In [85]:
# Calculate the percentage of missing values for each column
missing_percentage = (merged_data.isnull().sum() / len(merged_data)) * 100

# Display missing percentages in descending order
missing_percentage.sort_values(ascending=False)


MEXSTATE           72.452147
DEPE               62.643644
CANPROV            47.566587
DF                 33.702092
COMMODITY2         22.900870
USASTATE           14.455485
TRDTYPE             0.000000
DISAGMOT            0.000000
COUNTRY             0.000000
VALUE               0.000000
SHIPWT              0.000000
FREIGHT_CHARGES     0.000000
CONTCODE            0.000000
MONTH               0.000000
YEAR                0.000000
dtype: float64

#### Retaining High-Missing Columns

**MEXSTATE**, **DEPE**, and **CANPROV** have high missing values but are essential for geographic and operational context.  
Their missing entries are expected based on country or transport mode, so they should be **kept**, not removed.
-- The rest of the columns with missing values such as COOMMODITY2 are within an allowable range for missing values


In [103]:
# Save the merged_data DataFrame to a CSV file
merged_data.to_csv("merged_data_output.csv", index=False)


### The data visualisation will be done with PowerBi